### Authentication
First of all we authenticate in ee

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1ARtbsJoDdZzElAtxRos6DmdL0mbn8ATTaAhZxffbRziYiy_WC9fW9kb7pvc

Successfully saved authorization token.


In [326]:
# Get the desired zone shapefile
import json
with open('/home/aarja/Downloads/georeferences-main/shapefiles/totoral_las_lomitas/whole_farm/whole.geojson', 'r') as jf:
    fc = json.load(jf)
    totoral = ee.FeatureCollection(fc)

In [327]:
def clipper(image):
    global totoral
    return image.clip(totoral)

In [328]:
# Import the Sentinel-2 collection
#sentinel_2_msi = ee.ImageCollection("COPERNICUS/S2_SR").filterDate('2021-09-01', '2022-10-17').map(clipper)
sentinel_2_msi = ee.ImageCollection("COPERNICUS/S2").filterDate('2021-09-01', '2022-10-17').map(clipper)

In [329]:
def calculate_ndvi(image):
    ndvi = image.addBands(image.normalizedDifference(['B8', 'B4']))
    return ndvi

In [330]:
sentinel_2_msi = sentinel_2_msi.map(calculate_ndvi)

In [331]:
import folium

In [332]:
lat, lon = -30.749311, -64.057867

my_map = folium.Map(location=[lat, lon], zoom_start=22)

In [333]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [334]:
# Set visualization parameters for land cover.
vis_params = {
    'min': -1,'max': 1,
    'pallette': ['Red', 'Green']
}

vis_params = { 'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],}

SentinalNdvi = sentinel_2_msi.select('nd')
my_map.add_ee_layer(SentinalNdvi, vis_params, 'NDVI')

In [335]:
my_map.add_child(folium.LayerControl())
display(my_map)